In [2]:
import pandas as pd
import numpy as np

In [12]:
EQUATIONS = ['wave', 'gas_dynamics', 'brusselator', 'kuramoto_sivashinsky']
MODELS = ['feast', 'gat', 'gcn', 'point_gnn', 'point_net', 'point_transformer']

entries = []
for model in MODELS:
    for eq in EQUATIONS:
        test_loss = pd.read_csv(f"../output/csv_logs/{eq}/forecast/high/{model}/version_0/metrics.csv").test_loss.dropna().values[0]
        entries.append({'model': model, 'equation': eq, 'value': test_loss})

results = pd.DataFrame(entries)
results.pivot_table(values='value', index="model", columns="equation")




equation,brusselator,gas_dynamics,kuramoto_sivashinsky,wave
model,,,,
feast,0.000709,0.000587,0.000105,0.000031
gat,0.037057,0.042997,0.096714,0.011614
gcn,0.284355,0.181616,0.484141,0.039223
point_gnn,0.000164,0.000202,0.000031,0.000008
point_net,1.000867,0.171653,1.003212,0.988587
point_transformer,0.000557,0.000220,0.000213,0.000018
